In [1]:
# !pip install transformers --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pickle
import ast

import pandas as pd


from sklearn.model_selection import train_test_split


In [5]:
# brendanho123
# data_path = 'drive/My Drive/Colab Notebooks/W266/Final Project/data'

# bho1
data_path = 'drive/My Drive/Colab Notebooks/Final Project/data'

# bholly597
# data_path = 'drive/My Drive/Final Project/data'

# cwgiant
# data_path = '/content/drive/MyDrive/Final Project/data'

animes = pd.read_csv(f'{data_path}/raw_data/animes_final.csv')
profiles = pd.read_csv(f'{data_path}/raw_data/profiles_final.csv')
reviews = pd.read_csv(f'{data_path}/raw_data/reviews_final.csv')

In [6]:
animes.rename(columns={'uid': 'anime_uid'}, inplace=True)
animes.head()

,anime_uid,title,synopsis,episodes,members,popularity,ranked,score,Action,Adventure,...,1930s,1940s,1950s,1960s,1970s,1980s,1990s,2000s,2010s,2020s
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,25.0,489888,141,25.0,8.82,0,0,...,0,0,0,0,0,0,0,0,1,0
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,22.0,995473,28,24.0,8.83,0,0,...,0,0,0,0,0,0,0,0,1,0
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,13.0,581663,98,23.0,8.83,0,1,...,0,0,0,0,0,0,0,0,1,0
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",64.0,1615084,4,1.0,9.23,1,1,...,0,0,0,0,0,0,0,1,0,0
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,1.0,214621,502,22.0,8.83,1,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
animes['synopsis'] = animes['synopsis'].replace(r'\n', ' ', regex=True).replace(r'\r', ' ', regex=True)
animes['synopsis'].head(10)

0    Following their participation at the Inter-Hig...
1    Music accompanies the path of the human metron...
2    The Abyss—a gaping chasm stretching down into ...
3    "In order for something to be obtained, someth...
4    After helping revive the legendary vampire Kis...
5    Shigeo "Mob" Kageyama is now maturing and unde...
6    Stubborn, spoiled, and naïve, 10-year-old Chih...
7    Ever since the death of his father, the burden...
8    Following an encounter with oddity specialist ...
9    One year has passed since the Black Rebellion,...
Name: synopsis, dtype: object

In [8]:
len(animes)

16216

In [9]:
profiles.reset_index(inplace=True, names='profile_uid')
profiles.head()

,profile_uid,profile,favorites_anime_cleaned,Female,Male,Non-Binary,Unknown,birthday_epoch,age
0,0,DesolatePsyche,"[33352, 25013, 5530, 33674, 1482, 269, 18245, ...",0,1,0,0,781056000.0,30.0
1,1,baekbeans,"[11061, 31964, 853, 20583, 918, 9253, 34599, 3...",1,0,0,0,973814400.0,23.0
2,2,skrn,"[918, 2904, 11741, 17074, 23273, 32281, 9989, ...",0,0,0,1,NaN,NaN
3,3,edgewalker00,"[5680, 849, 2904, 3588, 37349]",0,1,0,0,NaN,NaN
4,4,aManOfCulture99,"[4181, 7791, 9617, 5680, 2167, 4382, 849, 235,...",0,1,0,0,941241600.0,24.0


In [10]:
reviews.rename(columns={'score': 'Overall_score'}, inplace=True)
reviews.head()

,uid,profile,anime_uid,text,Overall_score,Story_score,Animation_score,Sound_score,Character_score,Enjoyment_score
0,255938,DesolatePsyche,34096,"First things first. My ""reviews"" system is exp...",8,8,8,10,9,8
1,259117,baekbeans,34599,Let me start off by saying that Made in Abyss ...,10,10,10,10,10,10
2,253664,skrn,28891,"Art 9/10: It is great, especially the actions ...",7,7,9,8,8,8
3,8254,edgewalker00,2904,Story taking place 1 yr from where season 1 tr...,9,9,9,10,10,9
4,291149,aManOfCulture99,4181,Kyoto Animations greatest strength is being ab...,10,10,8,9,10,10


In [11]:
reviews['text'] = reviews['text'].replace(r'\n', ' ', regex=True).replace(r'\r', ' ', regex=True)
reviews['text'].head(10)

0    First things first. My "reviews" system is exp...
1    Let me start off by saying that Made in Abyss ...
2    Art 9/10: It is great, especially the actions ...
3    Story taking place 1 yr from where season 1 tr...
4    Kyoto Animations greatest strength is being ab...
5    When I first saw Code Geass I was like omgzxwt...
6    As someone who loves Studio Ghibli and its mov...
7    Code Geass R2 is sitting pretty on myanimelist...
8    code geass is one of those series that everybo...
9    Okay, so I was pleased to see that Clannad act...
Name: text, dtype: object

### Preprocess

In [12]:
reviews.sort_values(by=['profile'], inplace=True)
reviews.head()

,uid,profile,anime_uid,text,Overall_score,Story_score,Animation_score,Sound_score,Character_score,Enjoyment_score
115661,210287,-----noname-----,18441,Giant cliffhanger at the end of the season. Th...,2,1,6,6,1,4
16382,221727,-----noname-----,2025,I did not like this anime and I ended up dropp...,4,6,3,6,2,4
38672,208633,---SnowFlake---,1535,This in my opinion would have been a master pi...,6,4,6,6,8,4
111564,79296,---was-----,10110,The story is decent.This is an anime that you ...,8,7,10,9,7,9
32676,274580,--EYEPATCH--,35839,"To put it out there, I am very emotionally tou...",10,10,9,10,10,10


In [13]:
max_indices = reviews.groupby('profile')['uid'].idxmax()
filtered_reviews = reviews.loc[max_indices].reset_index(drop=True)
filtered_reviews.head()

,uid,profile,anime_uid,text,Overall_score,Story_score,Animation_score,Sound_score,Character_score,Enjoyment_score
0,221727,-----noname-----,2025,I did not like this anime and I ended up dropp...,4,6,3,6,2,4
1,208633,---SnowFlake---,1535,This in my opinion would have been a master pi...,6,4,6,6,8,4
2,79296,---was-----,10110,The story is decent.This is an anime that you ...,8,7,10,9,7,9
3,274580,--EYEPATCH--,35839,"To put it out there, I am very emotionally tou...",10,10,9,10,10,10
4,226341,--Mizu--,477,Mood & Atmosphere: If I can describe this anim...,10,10,10,10,10,10


In [14]:
users = filtered_reviews[['profile', 'text']].merge(profiles[['profile', 'favorites_anime_cleaned']], on='profile')#.reset_index(names='profile_uid')
users.head()

,profile,text,favorites_anime_cleaned
0,-----noname-----,I did not like this anime and I ended up dropp...,"[6774, 245, 2001, 11061, 16592, 1575, 21]"
1,---SnowFlake---,This in my opinion would have been a master pi...,"[2904, 6773, 10790]"
2,---was-----,The story is decent.This is an anime that you ...,[]
3,--EYEPATCH--,"To put it out there, I am very emotionally tou...",[]
4,--Mizu--,Mood & Atmosphere: If I can describe this anim...,"[21, 177, 6864, 4081, 5678, 23289]"


In [15]:
shuffled_users = users.sample(frac=1, random_state=42).reset_index(drop=True)

train_users, test_users = train_test_split(shuffled_users, test_size=0.2, random_state=42)
test_users, val_users = train_test_split(test_users, test_size=0.5, random_state=42)

In [16]:
train_users.head()

,profile,text,favorites_anime_cleaned
42293,ItachiHatesYou,Don't watch this. Here's my opinion why- The A...,[]
20376,oadziel,I rarely ever review anything I watch. Mostly ...,"[22199, 9253, 10620, 4181, 30654, 23755]"
45976,dregonskreper,I don't like writing or reading long reviews s...,"[28977, 9253, 934, 27991, 5081, 1530, 5643]"
1771,Erthar,"This is my first time doing a review, so bear ...","[4382, 9253, 10793, 2167, 4181]"
33113,erdem07,There are two problems with this anime. The fi...,"[4981, 5042, 30240, 3702]"


In [17]:
shows = animes[['anime_uid', 'title', 'synopsis']].sample(frac=1, random_state=42).reset_index(drop=True)
shows.head()

,anime_uid,title,synopsis
0,6097,Heisei Harenchi Gakuen,A modern-day version of Go Nagai's manga Haren...
1,6533,Atarashii Sekai,Official animated video to the Asian Kung-Fu G...
2,16782,Kotonoha no Niwa,"On a rainy morning in Tokyo, Takao Akizuki, an..."
3,6707,Kuroshitsuji II,The stage of Kuroshitsuji II opens on the li...
4,9366,Kaichou wa Maid-sama!: Omake dayo!,Extra episode included in Blu-ray and DVD volu...


In [18]:
"""
Anime features ---> Genres, popularity, Ranked/Score, episodes, Aired (start date)

popularity/members -> Choose 1 -> impute avg popularity
ranked/score -> Choose 1 -> impute avg score
episodes -> impute avg episodes count
aired_start_epoch -> impute avg start time and bucket

gender -> make unknown if not already
age -> impute avg age
"""

"""
Genres -> 'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons',
       'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical',
       'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military',
       'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance',
       'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai',
       'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports',
       'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri',

 -----> popularity
 -----> score
 -----> episodes
 -----> '1910s', '1920s', '1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s',
 -----> 'Female', 'Male','Non-Binary', 'Unknown'
 -----> age

"""
train_data.columns

NameError: name 'train_data' is not defined

In [ ]:
# def get_labels(users_df):
#   all_show_ids = set(shows['anime_uid'])

#   mapping_dict = {anime_uid: i for i, anime_uid in enumerate(shows['anime_uid'])}

#   one_hot_encoded = []
#   for row in users_df['favorites_anime_cleaned']:
#       one_hot_vector = [0] * len(all_show_ids)
#       for show_id in row:
#           if show_id in all_show_ids:
#               index = mapping_dict[show_id]
#               one_hot_vector[index] = 1
#       one_hot_encoded.append(one_hot_vector)

#   one_hot_df = pd.DataFrame(one_hot_encoded, columns=shows['anime_uid'].astype(str))
#   return one_hot_df


In [ ]:
# train_users_labels = get_labels(train_users)
# val_users_labels = get_labels(val_users)
# test_users_labels = get_labels(test_users)

In [ ]:
# train_users = train_users.drop(columns='favorites_anime_cleaned')
# test_users = test_users.drop(columns='favorites_anime_cleaned')
# val_users = val_users.drop(columns='favorites_anime_cleaned')

In [ ]:
# data_to_pickle = {
#     'train_users': train_users,
#     'train_labels': train_users_labels,
#     'test_users': test_users,
#     'test_labels': test_users_labels,
#     'val_users': val_users,
#     'val_labels': val_users_labels,
#     'shows': shows,
# }

# Save the data to a pickle file
# for key, value in data_to_pickle.items():
#     with open(f'{data_path}/{key}.pkl', 'wb') as f:
#         pickle.dump(value, f)

### Preprocess 2

In [19]:
data = reviews.merge(animes.rename(columns={'uid': 'anime_uid'}), on='anime_uid').merge(profiles, on='profile')
data.head()

,uid,profile,anime_uid,text,Overall_score,Story_score,Animation_score,Sound_score,Character_score,Enjoyment_score,...,2010s,2020s,profile_uid,favorites_anime_cleaned,Female,Male,Non-Binary,Unknown,birthday_epoch,age
0,210287,-----noname-----,18441,Giant cliffhanger at the end of the season. Th...,2,1,6,6,1,4,...,1,0,10861,"[6774, 245, 2001, 11061, 16592, 1575, 21]",0,0,0,1,1.577750e+09,4.0
1,221727,-----noname-----,2025,I did not like this anime and I ended up dropp...,4,6,3,6,2,4,...,0,0,10861,"[6774, 245, 2001, 11061, 16592, 1575, 21]",0,0,0,1,1.577750e+09,4.0
2,123215,Bear1211,18441,Blood Lad was a sad case adaption wise. Allott...,7,6,7,6,7,8,...,1,0,1557,"[2001, 32, 1210, 329, 457]",0,1,0,0,7.871040e+08,29.0
3,141893,Bear1211,226,"Elfen Lied is a textbook example of ""shock and...",4,4,5,6,3,4,...,0,0,1557,"[2001, 32, 1210, 329, 457]",0,1,0,0,7.871040e+08,29.0
4,122785,Bear1211,329,"Space, the final frontier, and what a frontier...",9,8,8,8,9,10,...,0,0,1557,"[2001, 32, 1210, 329, 457]",0,1,0,0,7.871040e+08,29.0


In [20]:
data['favorites_anime_cleaned'] = data['favorites_anime_cleaned'].apply(ast.literal_eval)

data['favorites_anime_cleaned'].iloc[0]

[6774, 245, 2001, 11061, 16592, 1575, 21]

In [24]:
len(data['anime_uid'].unique())

8113

In [22]:
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

train_data, test_data = train_test_split(shuffled_data, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [23]:
train_data.columns

Index(['uid', 'profile', 'anime_uid', 'text', 'Overall_score', 'Story_score',
       'Animation_score', 'Sound_score', 'Character_score', 'Enjoyment_score',
       'title', 'synopsis', 'episodes', 'members', 'popularity', 'ranked',
       'score', 'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons',
       'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical',
       'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military',
       'Music', 'Mystery', 'Parody', 'Police', 'Psychological', 'Romance',
       'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai',
       'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports',
       'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri',
       'aired_start_epoch', 'aired_end_epoch', 'anime_age',
       'aired_start_datetime', '1910s', '1920s', '1930s', '1940s', '1950s',
       '1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s',
       'profile_uid', 'favorites_an

In [ ]:
data_to_pickle = {
    'train_data': train_data,
    'test_data': test_data,
    'val_data': val_data,
}

In [ ]:
# Animes

# Score
def impute_score_with_mean(df):
    # Calculate the mean of the 'score' column
    score_mean = int(df['score'].mean())

    # Impute score with the average value
    df['score'].fillna(score_mean, inplace=True)

    return df

train_data = impute_score_with_mean(train_data)
test_data = impute_score_with_mean(test_data)
val_data = impute_score_with_mean(val_data)

# Eposide counts
def impute_episode_with_mean(df):
    # Calculate the mean of the 'episodes' column
    episode_mean = int(df['episodes'].mean())

    # Impute episodes with the average value
    df['episodes'].fillna(episode_mean, inplace=True)

    return df


train_data = impute_episode_with_mean(train_data)
test_data = impute_episode_with_mean(test_data)
val_data = impute_episode_with_mean(val_data)


# Impute start time bucket
def impute_decade(df):
    # Check if 'aired_start_epoch' column is null
    null_mask = df['aired_start_epoch'].isnull()

    # If 'aired_start_epoch' is null, set '2010s' column to 1
    df.loc[null_mask, '2010s'] = 1

    return df

train_data = impute_decade(train_data)
test_data = impute_decade(test_data)
val_data = impute_decade(val_data)



# User profiles

# Age
def impute_age_with_mean(df):

    # Calculate the mean of the 'age' column
    user_age_mean = int(df['age'].mean())

    # Impute age with the average value
    df['age'].fillna(user_age_mean, inplace=True)

    return df

train_data = impute_age_with_mean(train_data)
test_data = impute_age_with_mean(test_data)
val_data = impute_age_with_mean(val_data)

# No nulls in gender --> df[['Female','Male','Non-Binary','Unkown']]]


# Impute gender
def impute_gender(df):
    # Check if 'Unkown' column is null
    null_mask = df['Unknown'].isnull()

    # If 'Unkown' is null, set to 1 and other gender cols to 0
    df.loc[null_mask, 'Female'] = 0
    df.loc[null_mask, 'Male'] = 0
    df.loc[null_mask, 'Non-Binary'] = 0
    df.loc[null_mask, 'Unknown'] = 1

    return df

train_data = impute_gender(train_data)
test_data = impute_gender(test_data)
val_data = impute_gender(val_data)

In [ ]:
# Save the data to a pickle file
for key, value in data_to_pickle.items():
    with open(f'{data_path}/cleaned_data/{key}.pkl', 'wb') as f:
        pickle.dump(value, f)